![logo](../../_static/images/NCI_logo.png)


# Data Chunking with Dask


### Launch the Jupyter Notebook application

#### Using the public hh5 conda environment managed by CLEX

Many python modules are available under the hh5 conda environment that is maintained by CLEX, as well as additional modules such as that of CleF used in the previous examples. This environment is publically available and developed to service the CLEX users allowing use cases for the wider communinty.
```
    $ module use /g/data3/hh5/public/modules
    $ module load conda/analysis3-unstable
```  

Launch the Jupyter Notebook application:
```
    $ jupyter notebook
``` 

<div class="alert alert-info">
<b>NOTE: </b> This will launch the <b>Notebook Dashboard</b> within a new web browser window. 
</div>

#### Load the required modules

In [1]:
import xarray as xr
import netCDF4 as nc
%matplotlib inline

#### Opening Multiple Files at Once

xarray's `open_mfdataset` allows multiple files to be opened simultaneously.

In [2]:
!ls /g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701
path = '/g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701/*'

pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20350101-20541231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20550101-20741231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20750101-20941231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20950101-21001231.nc


### Exercise

In a similar way to open_dataset, use open_mfdataset to open the multiple files in `path` and have a look at the Dataset. Do you see anything different from the previous notebook?

<a href="#ans1" data-toggle="collapse">Answer</a>
<div class="collapse" id="ans1">
<pre><code>
f_ssp585 = xr.open_mfdataset(path)
f_ssp585
</code></pre>
</div>

### Chunks

Notice that it says:
`pr(time, lat, lon) float32 dask.array<shape=(31390, 180, 288), chunksize=(7300, 180, 288)`
There is now an additional component to the shape, and that is `chunksize`.

The chunking of the array comes from the integration of Dask with xarray. Dask (see: https://docs.dask.org/en/latest/) is a library for parallel computing. Dask divides the data array into small pieces called "chunks", with each chunk designed to be small enough to fit into memory. 

The file itself may be already chunked. Filesystem chunking is available in netCDF-4 and HDF5 datasets. CMIP6 data should all be netCDF-4 and include some form of chunking on the file.

In [3]:
!ncdump -hst '/g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701/pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231.nc'

netcdf pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231 {
dimensions:
	lat = 180 ;
	bnds = 2 ;
	lon = 288 ;
	time = UNLIMITED ; // (7300 currently)
variables:
	double lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:bounds = "lat_bnds" ;
		lat:standard_name = "latitude" ;
		lat:cell_methods = "time: point" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 180 ;
		lat:_DeflateLevel = 2 ;
		lat:_Shuffle = "true" ;
		lat:_Endianness = "little" ;
	double lat_bnds(lat, bnds) ;
		lat_bnds:long_name = "latitude bounds" ;
		lat_bnds:units = "degrees_north" ;
		lat_bnds:axis = "Y" ;
		lat_bnds:_Storage = "chunked" ;
		lat_bnds:_ChunkSizes = 180, 2 ;
		lat_bnds:_DeflateLevel = 2 ;
		lat_bnds:_Shuffle = "true" ;
		lat_bnds:_Endianness = "little" ;
	double lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:bounds = "lon_bnds" ;
		lon:standard_name = "longitude" ;
		lon:cell_methods = "time: poin

#### In this case the file is chunked such that  `pr:_ChunkSizes = 1, 180, 288 ;`

Here we see that the data is chunked in time, where one chunk is one time-step and all points in lat-lon.

![](images/Chunks.png)
image source: https://www.unidata.ucar.edu/blogs/developer/en/entry/chunking_data_why_it_matters

Consider 2 types of data access
1. Accessing a 2D lat-lon slice in time (RHS figure)
2. Accessing a time series at a single lat-lon point (LHS figure)

With the chunking above, the first type of data access only needs to access a single chunk, while the second type needs to access ALL the chunks of the data array regardless. This dataset will be fastest for 2D lat-lon single time-step data access.

In general, even without chunking - when the data is accessed contiguously (by index order) - time is the slowest variable to access, then y, with x being the fastest. With the chunking method of this CMIP6 dataset, time still remains the slowest variable. For more uniform variable access speeds more evenly spaced chunks would be needed, spacing the chunks in time, lat, and lon.

### Exercise

Time how long it takes to load the precipitation data at `time='2015-01-01'` and then time how long it takes to load the data at `lat=0` and `lon=180` (remember to use `method='nearest'` for the latter case). How much difference is there in the different access methods?

<a href="#ans4" data-toggle="collapse">Answer</a>
<div class="collapse" id="ans4">
<pre><code>
%%time
f_ssp585.pr.sel(time='2015-01-01').load()

-----------------------------------------------

%%time
f_ssp585.pr.sel(lat=0,lon=180,method='nearest').load()
</code></pre>
</div>

### Approximately the same amount of data took 100x longer to load.

The spatial dataset contains 51840 data-points and took order 100ms to load. The time-series dataset has 31390 data-points and took order 10,000 ms to load.

Chunking and the ways in which the data is read is important in considering both how you access the data and if you wish to parallelise your code.


### NetCDF file Chunks versus Dask Chunks

Keep in mind, dask chunking is different to chunking of the stored data. As we saw, the stored data is chunked with chunks of size (1,180,288). The Dask array was chunked with size (7300, 180, 288). You can change the chunking in the dask array. In the below example we are specifying that there be 730 chunks in time.

In [4]:
f_ssp585 = xr.open_mfdataset(path,chunks={'time':730})

f_ssp585

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 31390)
Coordinates:
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 dask.array<shape=(31390, 180, 2), chunksize=(7300, 180, 2)>
    lon_bnds   (time, lon, bnds) float64 dask.array<shape=(31390, 288, 2), chunksize=(7300, 288, 2)>
    pr         (time, lat, lon) float32 dask.array<shape=(31390, 180, 288), chunksize=(730, 180, 288)>
    time_bnds  (time, bnds) object dask.array<shape=(31390, 2), chunksize=(730, 2)>
Attributes:
    external_variables:     areacella
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               day
    activity_id:            ScenarioMIP
    branch_method:          standard

### How big do you make your chunks?

The rule of thumb for dask chunks is that you should "create arrays with a minimum chunksize of at least one million elements":  http://xarray.pydata.org/en/stable/dask.html#chunking-and-performance

netCDF4 chunks are often a lot smaller than dask array chunks. The minimum chunksize exists because if you have too many chunks, then queuing of operations when parallising will be slow, and if they are too big computation and memory can be wasted. The default chunks from dask gave us chunks of size: (7300, 180, 288) or nearly 400 million elements so we could try reducing those chunks if needed. The larger the array, the larger the cost of queueing and the larger chunks may be needed.

#### IMPORTANT: Whatever dask array chunks you use, make sure they align with the netCDF4 file chunks!!

So far our chunks have been in time, and the netCDF4 file is also chunked in time. If we tried to use dask chunks to optimise the time-series loading of data, it will not help! 

#### Exercise

Try it, load the data with chunks size `(31390,180,1)` (i.e. chunked in lon) and name that file `f_bad_chunk`. Try re-loading the time series of pr at `lat=0` and `lon=180` and time how long it takes.

<a href="#ans1" data-toggle="collapse">Answer</a>
<div class="collapse" id="ans1">
<pre><code>
f_bad_chunk = xr.open_mfdataset(path,chunks={'time':31390,'lat':180,'lon':1})
----------------------------------------
%%time
f_bad_chunk.pr.sel(lat=0,lon=180,method='nearest').load()
</code></pre>
</div>

What if we add some extra cores to the computation?

You can easily parallelise xarray code using the dask.distributed.Client and dask array calculations will be run in parallel.

In [ ]:
from dask.distributed import Client
c = Client()
c

#### Exercise

Try running your previous code for `f_bad_chunk` again loading the time series of pr at `lat=0` and `lon=180` and time how long it takes now that there are 4 workers in the dask cluster.

Do the same with the original chunking method of `f_ssp585` and see if there is a difference.

<a href="#ans2" data-toggle="collapse">Answer</a>
<div class="collapse" id="ans2">
<pre><code>
%%time
f_bad_chunk.pr.sel(lat=0,lon=180,method='nearest').load()

----------------------------------------

%%time
f_ssp585.pr.sel(lat=0,lon=180,method='nearest').load()
</code></pre>
</div>

### Poor chunking with dask can make your performance worse!

Both the size of the chunks and the alignment of the chunks with the filesystem chunks are imporant to keep in mind when creating dask chunks. 